In [ ]:
# Organize the data
# Sort missing values
# Change to numerical
# Split the data
# Instantiate the model
# Make predictions

In [ ]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from joblib import dump, load

In [ ]:
data = pd.read_csv("car-sales-extended-missing-data.csv")
data.dtypes

In [ ]:
data.dropna(subset="Price", inplace=True)

In [ ]:
data.isna().sum()

In [ ]:
X = data.drop("Price", axis=1)
y = data["Price"]

In [ ]:
cat_features = ["Make", "Colour"]
cat_imputer = SimpleImputer(strategy="constant", fill_value="missing")

door_features = ["Doors"]
door_imputer = SimpleImputer(strategy="constant", fill_value=4)

odo_features = ["Odometer (KM)"]
odo_imputer = SimpleImputer(strategy="mean")

one_hot = OneHotEncoder()

transformer = ColumnTransformer([
    ("cat", cat_imputer, cat_features),
    ("door", door_imputer, door_features),
    ("odo", odo_imputer, odo_features)
])

In [ ]:
transformed_X = transformer.fit_transform(X)

In [ ]:
X_df = pd.DataFrame(data=transformed_X, columns=["Make", "Colour", "Doors", "Odometer (KM)"])

In [ ]:
X_df.head()

In [ ]:
one_hot = OneHotEncoder()

column_data = ["Make", "Colour", "Doors", "Odometer (KM)"]
transformer = ColumnTransformer([
    ("one_hot", one_hot, column_data)
], sparse_threshold=0, remainder="passthrough")

In [ ]:
transformed_X = transformer.fit_transform(X_df)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(transformed_X, y, test_size=0.2)
model = RandomForestRegressor(n_jobs=-1).fit(X_train, y_train)

In [ ]:
model.score(X_test, y_test)

In [ ]:
model.get_params()

In [ ]:
X_df.isna().sum()

In [ ]:
param_distributions = {
    "n_estimators": [100, 400, 700, 1000, 1200],
    "max_depth": [2, 6, 10, 14],
    "min_samples_split": [2, 4, 6, 8, 10],
    "min_samples_leaf": [2, 4, 6, 8, 10],
    "max_features": [2, 4, 6, 8, 10],
}

rs_model = RandomizedSearchCV(model, param_distributions, n_iter=20, cv=5, verbose=2).fit(X_test, y_test)

In [ ]:
rs_model.best_params_

In [ ]:
rs_model.score(X_test, y_test)

In [ ]:
dump(rs_model, filename="test.joblib")

# Try using Pipline()

In [15]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from joblib import dump, load

In [16]:
data = pd.read_csv("car-sales-extended-missing-data.csv")
data.isna().sum()

Make             49
Colour           50
Odometer (KM)    50
Doors            50
Price            50
dtype: int64

In [17]:
data.dropna(subset="Price", inplace=True)
data.isna().sum()

Make             47
Colour           46
Odometer (KM)    48
Doors            47
Price             0
dtype: int64

In [23]:
cat_features = ["Make", "Colour"]
cat_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("one_hot", OneHotEncoder(handle_unknown="ignore"))
])

door_feature = ["Doors"]
door_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value=4))
])

odometer_feature = ["Odometer (KM)"]
odometer_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean"))
])

In [24]:
preprocessor = ColumnTransformer(transformers=[
    ("cat_imputer", cat_transformer, cat_features),
    ("door_imputer", door_transformer, door_feature),
    ("odometer_imputer", odometer_transformer, odometer_feature)
])

In [25]:
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", RandomForestRegressor(n_jobs=-1))
])

In [26]:
X = data.drop("Price", axis=1)
y = data["Price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [27]:
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat_imputer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value='missing',
                                                                                 strategy='constant')),
                                                                  ('one_hot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Make', 'Colour']),
                                                 ('door_imputer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(fill_value=4,
                                                                                 strategy='constant'))]),
                                                  ['Doors']),
                                                 ('odometer_imputer',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer())]),
                                                  ['Odometer (KM)'])])),
                ('model', RandomForestRegressor(n_jobs=-1))])

In [28]:
model.score(X_test, y_test)

0.23265354026313334